In [1]:
import os
from roboflow import Roboflow
from PIL import Image
from IPython.display import display
import json
import matplotlib.pyplot as plt
import cv2

import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np

In [2]:
current_dir = (os.getcwd())
current_dir

'd:\\DAFFA PINJEM BUAT BACKUP\\TA GASS\\FIX\\Yolo'

In [3]:
source_dir = './Multi-Object/'
target_dir = './Multi-Object-Result/'
predict_dir = './Predict/'

In [4]:
# Initialize API for coffee bean detection
rf = Roboflow(api_key="IluK8shPQ2JbRq6f5FVM")

# Reference : https://universe.roboflow.com/coffe-dataset/coffee-beans-znwfe/model/2
project = rf.workspace().project("coffee-beans-znwfe")
model = project.version(2).model
confident = 40
overlap = 30

def coffee_detection(filename):
  # infer on a local image
  result = model.predict(f"{source_dir}{filename}", confidence=confident, overlap=overlap).json()

  # visualize your prediction
  # model.predict(f"{source_dir}{filename}", confidence=confident, overlap=overlap).save(f"{target_dir}{filename}")
  return result

loading Roboflow workspace...
loading Roboflow project...


In [5]:
def get_pretrained_model(num_classes):
    model = models.resnet50(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, num_classes)  # Ganti num_classes dengan jumlah kelas dalam dataset Anda
    return model

In [6]:
def load_model(checkpoint_path, model):
    model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))
    model.eval()
    return model

In [7]:
resnet50 = get_pretrained_model(2)

defect_label_list = ["Defect", "Normal"]
defect_path = 'resnet50-coffee-defect.pth'
defect_model = load_model(defect_path, resnet50)

resnet50 = get_pretrained_model(4)

roast_label_list = ['Dark', 'Green', 'Light', 'Medium']
roastlevel_path = 'resnet50-coffee-roast-level.pth'
roastlevel_model = load_model(roastlevel_path, resnet50)

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.


In [8]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize( [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [9]:
file_images = os.listdir(source_dir)
img_list = []

for file_image in file_images:
    result = coffee_detection(file_image)

    # Open the image file
    for index, predict in enumerate(result["predictions"]):
        img = Image.open(os.path.join(source_dir, file_image))

        # Set the crop region
        x, y, width, height = predict["x"], predict["y"], predict["width"], predict["height"]
        max_width = max(width, height)
        crop_region = (x-(int(max_width/2)), y-(int(max_width/2)), x+(int(max_width/2)), y+(int(max_width/2)))

        # Crop the image
        cropped_img = img.crop(crop_region)

        # Save cropped image
        cropped_filename = f"{file_image.split('.')[0]}-{index}.jpg"
        cropped_img.save(f"./Predict/{cropped_filename}")
        img_tuple = (cropped_filename, file_image, (x, y, width, height))
        img_list.append(img_tuple)

        # Display the original and cropped images
        # fig, (ax1, ax2) = plt.subplots(1, 2)
        # ax1.imshow(img)
        # ax1.set_title('Original Image')
        # ax2.imshow(cropped_img)
        # ax2.set_title('Cropped Image')
        # plt.show()

print(img_list)

# with open('result.json', 'w') as f:
#     json.dump(result_list, f)

[('Defect 100%-0.jpg', 'Defect 100%.JPG', (1848, 2043, 639, 448)), ('Defect 100%-1.jpg', 'Defect 100%.JPG', (1171, 1686, 548, 448)), ('Defect 100%-2.jpg', 'Defect 100%.JPG', (1420, 2346, 548, 440)), ('Defect 100%-3.jpg', 'Defect 100%.JPG', (847, 1154, 581, 415)), ('Defect 100%-4.jpg', 'Defect 100%.JPG', (847, 2035, 548, 398)), ('Defect 100%-5.jpg', 'Defect 100%.JPG', (2583, 1948, 548, 390)), ('Defect 100%-6.jpg', 'Defect 100%.JPG', (1998, 1428, 490, 398)), ('Defect 100%-7.jpg', 'Defect 100%.JPG', (2720, 1237, 456, 498)), ('Defect 100%-8.jpg', 'Defect 100%.JPG', (2081, 847, 606, 398)), ('Defect 50% Normal 50%-0.jpg', 'Defect 50% Normal 50%.JPG', (1827, 992, 498, 540)), ('Defect 50% Normal 50%-1.jpg', 'Defect 50% Normal 50%.JPG', (2587, 731, 606, 465)), ('Defect 50% Normal 50%-2.jpg', 'Defect 50% Normal 50%.JPG', (2592, 2234, 548, 448)), ('Defect 50% Normal 50%-3.jpg', 'Defect 50% Normal 50%.JPG', (1183, 801, 606, 506)), ('Defect 50% Normal 50%-4.jpg', 'Defect 50% Normal 50%.JPG', (2700,

In [10]:
# loop through the images and make predictions
for img_predict in img_list:
    # load the image and apply the transformations
    image = Image.open(os.path.join(predict_dir, img_predict[0]))
    image = transform(image).unsqueeze(0)

    # make a prediction
    with torch.no_grad():
        defect_output = defect_model(image)
        # get the predicted class label
        predicted_defect = np.argmax(defect_output.numpy())
        defect_label = defect_label_list[predicted_defect.item()]

        roast_output = roastlevel_model(image)
        # get the predicted class label
        predicted_roast = np.argmax(roast_output.numpy())
        roast_label = roast_label_list[predicted_roast.item()]

        if not os.path.exists(os.path.join(target_dir, img_predict[1])):
            img_result = cv2.imread(os.path.join(source_dir, img_predict[1]))
        else:
            img_result = cv2.imread(os.path.join(target_dir, img_predict[1]))

        x, y, w, h = img_predict[2]

        # Draw bounding box on the image
        cv2.rectangle(img_result, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)), (0, 0, 255), 2)

        # Add label to the bounding box
        cv2.putText(img_result, f"{roast_label} - {defect_label}", (x - int(w/2), y - int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Save the img_result with bounding box and label
        cv2.imwrite(os.path.join(target_dir, img_predict[1]), img_result)
